In [7]:
# GIS-analyse av bhg i Oppdal - eksempel på anvendelse av kart 

import pandas as pd
import geopandas as gpd

from shapely.geometry import Point

import streamlit as st
import folium
from streamlit_folium import st_folium

RuntimeError: Runtime hasn't been created!

In [6]:
# !pip3 install geopandas
# !pip3 install streamlit
# !pip3 install streamlit_folium
# !pip3 install folium


In [8]:
pd.__version__

'1.5.3'

In [9]:
# df = pd.read_csv('data.csv', encoding='UTF-8',index_col='Id
df = pd.read_excel('data.xlsx')

In [10]:
df.head()

,Id,Barnehage,UTM_X,UTM_Y,Ant_barn,Ant_ansatte,Adresse,Jernbanestasjonen
0,1,Pikhaugen,9.669206,62.591338,35,8,"Pionvegen 1, 7340 Oppdal","'62.5931427','9.6929923'"
1,2,Høgmo barnehage,9.697422,62.599715,40,10,"Gorsetvegen 15, 7340 Oppdal","'62.5931427','9.6929923'"
2,3,Bjerkehagen,9.691496,62.597591,30,7,"Luvegen 5, 7340 Oppdal","'62.5931427','9.6929923'"
3,4,Brennan,9.670819,62.589220,50,12,"Soleieveien 30, 7340 Oppdal","'62.5931427','9.6929923'"


In [11]:
# Convert UTM coordinates to Point geometry
df['geometry'] = [Point(xy) for xy in zip(df.UTM_Y, df.UTM_X)]
# gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude))

# Create a GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry='geometry')
gdf.set_crs(epsg=4326, inplace=True)

,Id,Barnehage,UTM_X,UTM_Y,Ant_barn,Ant_ansatte,Adresse,Jernbanestasjonen,geometry
0,1,Pikhaugen,9.669206,62.591338,35,8,"Pionvegen 1, 7340 Oppdal","'62.5931427','9.6929923'",POINT (62.59134 9.66921)
1,2,Høgmo barnehage,9.697422,62.599715,40,10,"Gorsetvegen 15, 7340 Oppdal","'62.5931427','9.6929923'",POINT (62.59972 9.69742)
2,3,Bjerkehagen,9.691496,62.597591,30,7,"Luvegen 5, 7340 Oppdal","'62.5931427','9.6929923'",POINT (62.59759 9.6915)
3,4,Brennan,9.670819,62.589220,50,12,"Soleieveien 30, 7340 Oppdal","'62.5931427','9.6929923'",POINT (62.58922 9.67082)


In [12]:
df['voksentetthet'] = df['Ant_ansatte']/df['Ant_barn']

In [13]:
import folium

# Create a map object centered around an average location
map_center = [df['UTM_Y'].mean(), df['UTM_X'].mean()]
mymap = folium.Map(location=map_center, zoom_start=12)

# Add markers to the map
for idx, row in df.iterrows():
    folium.Circle(
        location=[row['UTM_Y'], row['UTM_X']],
        radius = row['Ant_barn']*5,
            color = 'blue',
            fill_color = 'blue',
            fill = True,          
        popup=f"{row['Barnehage']}<br>Kids: {row['Ant_barn']}<br>Employees: {row['Ant_ansatte']}"
    ).add_to(mymap)

# Display the map
mymap




In [14]:
df

,Id,Barnehage,UTM_X,UTM_Y,Ant_barn,Ant_ansatte,Adresse,Jernbanestasjonen,geometry,voksentetthet
0,1,Pikhaugen,9.669206,62.591338,35,8,"Pionvegen 1, 7340 Oppdal","'62.5931427','9.6929923'",POINT (62.5913379 9.6692059),0.228571
1,2,Høgmo barnehage,9.697422,62.599715,40,10,"Gorsetvegen 15, 7340 Oppdal","'62.5931427','9.6929923'",POINT (62.599715 9.6974217),0.250000
2,3,Bjerkehagen,9.691496,62.597591,30,7,"Luvegen 5, 7340 Oppdal","'62.5931427','9.6929923'",POINT (62.597591 9.6914961),0.233333
3,4,Brennan,9.670819,62.589220,50,12,"Soleieveien 30, 7340 Oppdal","'62.5931427','9.6929923'",POINT (62.5892201 9.6708194),0.240000


In [15]:
def get_color(ratio):
    if ratio > 0.245:
        return 'green'
    elif 0.229 <= ratio < 0.245:
        return 'orange'
    else:
        return 'red'

In [16]:
map_center = [df['UTM_Y'].mean(), df['UTM_X'].mean()]

# mymap = folium.Map(location=map_center, zoom_start=12)
mymap = folium.Map(location=[62.5931427,9.6929923], zoom_start=12)

# Add circles with colors
for idx, row in df.iterrows():
    folium.Circle(
        location=[row['UTM_Y'], row['UTM_X']],
        popup=f"{row['Barnehage']}<br>Barn: {row['Ant_barn']}<br>Ansatte: {row['Ant_ansatte']}<br>Voksentetthet: {row['voksentetthet']:.2f}",
        radius=row['Ant_barn'] * 10,  # Adjust as necessary
        color=get_color(row['voksentetthet']),
        fill=True,
        fill_color=get_color(row['voksentetthet'])
    ).add_to(mymap)

# Display the map
mymap

In [30]:
# gdf

In [17]:
ref_point = df['Jernbanestasjonen'].iloc[0]

In [17]:
# # Assuming your DataFrame is already converted to a GeoDataFrame 'gdf'
# # and both kindergarten and reference point locations are in WGS84 (EPSG:4326)

# ref_point.set_crs(epsg=4326, inplace=True)
# # Reproject to a suitable UTM zone CRS

# ref_point = ref_point.to_crs(epsg=4326, inplace=True)


In [18]:
ref_point_lon = '9.6929923'
ref_point_lat = '62.5931427'

ref_point = Point(ref_point_lon, ref_point_lat)

In [19]:

# Convert kindergarten data to GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.UTM_X, df.UTM_Y))

# Set CRS to WGS84 (if it's not already)
gdf = gdf.set_crs(epsg=4326)

# Find a suitable UTM zone CRS for your region and reproject
gdf = gdf.to_crs(epsg=32632)
ref_point = gpd.GeoSeries([ref_point], crs=4326).to_crs(epsg=32632)[0]


In [20]:
df['AvstandJernbane'] = gdf['geometry'].distance(ref_point)

In [21]:
df

,Id,Barnehage,UTM_X,UTM_Y,Ant_barn,Ant_ansatte,Adresse,Jernbanestasjonen,geometry,voksentetthet,AvstandJernbane
0,1,Pikhaugen,9.669206,62.591338,35,8,"Pionvegen 1, 7340 Oppdal","'62.5931427','9.6929923'",POINT (62.5913379 9.6692059),0.228571,1238.072002
1,2,Høgmo barnehage,9.697422,62.599715,40,10,"Gorsetvegen 15, 7340 Oppdal","'62.5931427','9.6929923'",POINT (62.599715 9.6974217),0.250000,766.749567
2,3,Bjerkehagen,9.691496,62.597591,30,7,"Luvegen 5, 7340 Oppdal","'62.5931427','9.6929923'",POINT (62.597591 9.6914961),0.233333,501.516166
3,4,Brennan,9.670819,62.589220,50,12,"Soleieveien 30, 7340 Oppdal","'62.5931427','9.6929923'",POINT (62.5892201 9.6708194),0.240000,1219.784744


In [22]:
st.title('Kartanalyse barnehager')
# folium_static(mymap)
st_map = st_folium(mymap)

st.write(mymap)
st.write("## session_state")
st.write(st.session_state)

2024-09-10 13:49:29.123 
  command:

    streamlit run /Users/jprestmo001/Library/Python/3.12/lib/python/site-packages/ipykernel_launcher.py [ARGUMENTS]


NameError: name 'st_folium' is not defined